In [ ]:
#gameId = "doapp"
#gameId = "sfiii3n"
#gameId = "umk3"
gameId = "tektagt"

In [ ]:
import sys, os
import time
import cv2
import tensorflow as tf

timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

sys.path.append(os.path.join(os.path.abspath(''), '../../games/'))   
sys.path.append(os.path.join(os.path.abspath(''), '../../utils'))
sys.path.append(os.path.join(os.path.abspath(''), '../../pythonGamePadInterface'))
                                                                                                                                 
from diambraGamepad import diambraGamepad 
from policies import gamepadPolicy, RLPolicy # To train AI against another AI or HUM
from makeDiambraEnv import *

In [ ]:
# Common settings
diambraKwargs = {}
diambraKwargs["roms_path"] = "../../roms/mame/"
diambraKwargs["binary_path"] = "../../customMAME/"
diambraKwargs["frame_ratio"] = 6
diambraKwargs["throttle"] = False
diambraKwargs["sound"] = diambraKwargs["throttle"]

diambraKwargs["player"] = "P2" # 1P
#diambraKwargs["player"] = "P1P2" # 2P

#keyToAdd = None
keyToAdd = []
keyToAdd.append("actionsBuf")

if gameId != "tektagt": # DOA, SFIII, UMK3
    keyToAdd.append("ownHealth")
    keyToAdd.append("oppHealth")
else: # TEKTAG
    keyToAdd.append("ownHealth_1")
    keyToAdd.append("ownHealth_2")
    keyToAdd.append("oppHealth_1")
    keyToAdd.append("oppHealth_2")

keyToAdd.append("ownPosition")
keyToAdd.append("oppPosition")
#keyToAdd.append("ownWins")
#keyToAdd.append("oppWins")
keyToAdd.append("stage")
keyToAdd.append("character")

In [ ]:
if gameId != "tektagt":
    diambraKwargs["characters"] = ["Ryu", "Ken"]
else:
    diambraKwargs["characters"] = [["Random", "Random"], ["Random", "Kuma"]]
diambraKwargs["charOutfits"] = [2, 2]

In [ ]:
# GamePad policy initialization
gamePad_policy = gamepadPolicy(diambraGamepad)

In [ ]:
# DIAMBRA gym kwargs
diambraGymKwargs = {}
diambraGymKwargs["P2brain"] = None#gamePad_policy
diambraGymKwargs["continue_game"] = 0.0
diambraGymKwargs["show_final"] = False
diambraGymKwargs["gamePads"] = [None, diambraGymKwargs["P2brain"]]
diambraGymKwargs["actionSpace"] = ["discrete", "multiDiscrete"]
diambraGymKwargs["attackButCombinations"] = [True, False]

In [ ]:
# Recording kwargs
trajRecKwargs = {}                                                          
trajRecKwargs["user_name"] = "Alex"
trajRecKwargs["file_path"] = os.path.join( "~/DIAMBRA/trajRecordings", gameId)
trajRecKwargs["ignore_p2"] = 0                                    
trajRecKwargs["commitHash"] = "0000000"

In [ ]:
trajRecKwargs = None

In [ ]:
wrapperKwargs = {}
wrapperKwargs["hwc_obs_resize"] = [256, 256, 1]
wrapperKwargs["normalize_rewards"] = True
wrapperKwargs["clip_rewards"] = False
wrapperKwargs["frame_stack"] = 6
wrapperKwargs["dilation"] = 1
wrapperKwargs["scale"] = True
wrapperKwargs["scale_mod"] = 0

numEnv=1

envId = gameId + "_Test"
env = make_diambra_env(diambraMame, env_prefix=envId, num_env=numEnv, seed=timeDepSeed, 
                       diambra_kwargs=diambraKwargs, diambra_gym_kwargs=diambraGymKwargs,
                       wrapper_kwargs=wrapperKwargs, key_to_add=keyToAdd, no_vec=True,
                       traj_rec_kwargs=trajRecKwargs)

In [ ]:
print("Obs space =", env.observation_space)
print("Obs space type =", env.observation_space.dtype)
print("Obs space high bound =", env.observation_space.high)
print("Obs space low bound =", env.observation_space.low)

In [ ]:
# Printing action spaces
for idx in range(2):
    
    if diambraKwargs["player"] != "P1P2" and idx == 1:
        continue
        
    print("Action space P{} = ".format(idx+1), env.action_spaces[idx])
    print("Action space type P{} = ".format(idx+1), env.action_spaces[idx].dtype)
    if diambraGymKwargs["actionSpace"][idx] == "multiDiscrete":
        print("Action space n = ", env.action_spaces[idx].nvec)
    else:
        print("Action space n = ", env.action_spaces[idx].n)

In [ ]:
observation = env.reset()

shp = observation.shape

print("Number of characters =", len(env.charNames))

additionalParP1 = int(observation[0,0,shp[2]-1])
print("Additional Parameters P1 =", additionalParP1)

nScalarAddParP1 = additionalParP1 - len(env.charNames)\
                - env.actBufLen*(env.n_actions[0][0]+env.n_actions[0][1]) # 1P
print("Number of scalar Parameters P1 =", nScalarAddParP1)


if diambraKwargs["player"] == "P1P2": 
    additionalParP2 = int(observation[128,0,shp[2]-1])
    print("Additional Parameters P2 =", additionalParP2)
    
    nScalarAddParP2 = additionalParP2 - len(env.charNames)\
                    - env.actBufLen*(env.n_actions[1][0]+env.n_actions[1][1])# 2P
    print("Number of scalar Parameters P2 =", nScalarAddParP2)
    
limAct = [None, None]
for idx in range(2):
    limAct[idx] = [env.actBufLen * env.n_actions[idx][0], 
                   env.actBufLen * env.n_actions[idx][0] + env.actBufLen * env.n_actions[idx][1]]

In [ ]:
cumulativeEpRew = 0.0
cumulativeEpRewAll = []

maxNumEp = 100
currNumEp = 0

#sess = tf.Session();

while currNumEp < maxNumEp:

    # 1P
    #action = [0, 0]
    action = env.action_spaces[0].sample()
    
    # 2P
    action2 = env.action_spaces[1].sample()
    if diambraKwargs["player"] == "P1P2":
        action = np.append(action, action2)

    #action = int(input("Action"))
    print("Action:", action)
    observation, reward, done, info = env.step(action)
    
    #if reward != 0:
    #    print("Reward =", info["rewards"])
    #    input("AAA")
    
    doit = True
    if info["round_done"] == True or doit:

        addPar = observation[:,:,shp[2]-1]
        addPar = np.reshape(addPar, (-1))
        
        # P1
        addParP1 = addPar[1:additionalParP1+1]
        actionsP1 = addParP1[0:additionalParP1-nScalarAddParP1-env.numberOfCharacters]
        
        moveActionsP1   = actionsP1[0:limAct[0][0]]
        attackActionsP1 = actionsP1[limAct[0][0]:limAct[0][1]]
        moveActionsP1   = np.reshape(moveActionsP1, (env.actBufLen,-1))
        attackActionsP1 = np.reshape(attackActionsP1, (env.actBufLen,-1))
        print("Move actions P1 =\n", moveActionsP1)
        print("Attack actions P1 =\n ", attackActionsP1)
        #input("Pausa1")
        
        othersP1 = addParP1[additionalParP1-nScalarAddParP1-env.numberOfCharacters:]
        print("ownHealthP1 = ", othersP1[0])
        print("oppHealthP1 = ", othersP1[1])
        print("ownPositionP1 = ", othersP1[2])
        print("oppPositionP1 = ", othersP1[3])
        print("stageP1 = ", othersP1[4])
        print("Playing Char P1 = ", env.charNames[list(othersP1[nScalarAddParP1:
                                                                nScalarAddParP1 + env.numberOfCharacters]).index(1.0)])
        
        # 2P
        if diambraKwargs["player"] == "P1P2":
            addParP2 = addPar[int((shp[0]*shp[1])/2)+1:int((shp[0]*shp[1])/2)+additionalParP2+1]
            actionsP2 = addParP2[0:additionalParP2-nScalarAddParP2-env.numberOfCharacters]
            
            moveActionsP2   = actionsP2[0:limAct[1][0]]
            attackActionsP2 = actionsP2[limAct[1][0]:limAct[1][1]]
            moveActionsP2   = np.reshape(moveActionsP2, (env.actBufLen,-1))
            attackActionsP2 = np.reshape(attackActionsP2, (env.actBufLen,-1))
            print("Move actions P2 =\n", moveActionsP2)
            print("Attack actions P2 =\n", attackActionsP2)
            #input("Pausa1")        
        
            othersP2 = addParP2[additionalParP2-nScalarAddParP2-env.numberOfCharacters:]
            print("ownHealthP2 = ", othersP2[0])
            print("oppHealthP2 = ", othersP2[1])
            print("ownPositionP2 = ", othersP2[2])
            print("oppPositionP2 = ", othersP2[3])
            print("stageP2 = ", othersP2[4])
            print("Playing Char P2 = ", env.charNames[list(othersP2[nScalarAddParP2:
                                                                    nScalarAddParP2 + env.numberOfCharacters]).index(1.0)])

        #input("Pausa1")
        
        #print(np.array(observation).astype(np.float32).shape)
        #input("Pausa2")
        #print(tf.cast(observation, tf.float32).eval(session=sess))
   
    
    #if info["round_done"] == True or doit:
    #    tensor = tf.cast(observation, tf.float32)
    #    tensor2 = tensor[:,:,shp[2]-1]
    #    print("Number off additional param = ", tensor2[0,0].eval(session=sess))    
    #    tensor2 = tf.reshape(tensor2, [-1]) 
    #    #print("After reshaping = ", tensor2)    
    #    tensor2 = tensor2[1:1+additionalPar]
    #    #print("Tensor=", tensor)
    #    tensor_actions = tf.reshape(tensor2[0:additionalPar-4], [12,-1]) 
    #    print("Tensor=", tensor2[additionalPar-4:additionalPar+1].eval(session=sess))
    #    print("Tensor=", tensor_actions.eval(session=sess))
    #    input("Pausa")
        
    obs = np.array(observation).astype(np.float32)
    
    for idx in range(shp[2]-1):
        cv2.imshow("image"+str(idx), obs[:,:,idx])
    
    cv2.waitKey()
    
    #print("Frames shape:", observation.shape)
    #print("Reward:", reward)
    #print("Fighting = ", info["fighting"])
    #print("Rewards = ", info["rewards"])
    #print("HealthP1 = ", info["healthP1"])
    #print("HealthP2 = ", info["healthP2"])
    #print("HealthP1_1 = ", info["healthP1_1"])
    #print("HealthP1_2 = ", info["healthP1_2"])
    #print("HealthP2_1 = ", info["healthP2_1"])
    #print("HealthP2_2 = ", info["healthP2_2"])
    #print("PositionP1 = ", info["positionP1"])
    #print("PositionP2 = ", info["positionP2"])
    #print("WinP1 = ", info["winsP1"]) 
    #print("WinP2 = ", info["winsP2"])

    
    cumulativeEpRew += reward
    
    if np.any(done):
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        sys.stdout.flush()
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeEpRew = 0.0

        observation = env.reset()
        input("Stop")

print("Mean cumulative reward = ", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward = ", np.std(cumulativeEpRewAll))       
    
env.close()